In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.ops import rnn, rnn_cell
import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy as np
import os
import time

NUM_EPOCHS = 500  # 반복 횟수
chunk_size = 1  # 한 개 요소의 크기
# n_chunks = 7 #sequence_length
rnn_size = 25  # rnn cell의 갯수
PH = 5  # PH


# # numpy array인 ndarray에는 append 함수가 없어서 vstack이나 concatenate함수를 이용해서 정의
# def npAppend(x, y):
#     x = np.vstack((x, y))
#     return x


In [2]:

# readData reads data from the specified pre-processed input data file.
# The function returns an array of input data points and an array of the
# corresponding desired outputs.
def readData(filePath):
    allList = []
    x_data = []
    y_data = []
    newPointx = []
    newPointy = []
    with open(filePath, 'r') as f:
        for line in f:
            allList.append(float(line))

    df = pd.Series(allList)  # 일차원 리스트를 pandas 데이터프레임화
    while True:
        for i in df[0:7]:
            newPointx.append([float(i)])  # 데이터프레임 앞의 7개를 newPointx 리스트에 삽입
        newPointy.append(float(df[6 + (PH / 5)]))  # 데이터프레임 그 다음(10번째)을 newPointy 리스트에 삽입
        x_data.append(newPointx)  # x_data 리스트에 newPointx 리스트를 삽입 (x_data는 array of array가 됨)
        y_data.append(newPointy)  # 위와 동일

        newPointx = []  # 다음 반복을 위해 newPointx,y를 빈 리스트로 초기화
        newPointy = []
        df = df.shift(-1)  # 데이터프레임 왼 쪽으로 1칸 쉬프트
        if (math.isnan(df[6 + (PH / 5)])):  # 만약 8번째 데이터프레임이 NaN(Not a Number) 라면 반복 중지
            break
    x_data = np.asarray(x_data)
    y_data = np.asarray(y_data)
    data = [x_data, y_data]

    return data;

In [3]:
start_time = time.time()

# 실제 데이터 사용 중지
'''
trainFileName = [85655,209019,365303,485709,553778,573060,579883,822250,839654,885633,895646,920087]
testFileName = [1007000,1185429,1393413,3008387]
'''
gitTrainFileName = []
gitTestFileName = []

trainData_in_list = []
trainData_out_list = []

testData_in_list = []
testData_out_list = []

trainFileName = []
testFileName = []

# git_input_data 폴더 밑에 12_test.csv, 12_train.csv 처럼 파일 있으니, 이 이름들을 배열에 저장시켜야 나중에 편해짐
gitFileList = os.listdir("git_input_data")
for fn in gitFileList:
    if fn.find("test") != -1:  # 파일이름에 test가 들어간다면
        gitTestFileName.append(fn)
    elif fn.find("train") != -1:  # 파일 이름에 train이 들어간다면
        gitTrainFileName.append(fn)

In [4]:
#uci-preprocessed 폴더 밑에 preprocessed-23.csv 처럼 파일 있으니, 이 이름들을 배열에 저장
#1번~56번은 학습용, 57번~70번은 시험용에 분리
uciFileList = os.listdir("uci-preprocessed")
count=1
for fn in uciFileList:
    if(count>=1 and count<=56):
        trainFileName.append(fn)
    else:
        testFileName.append(fn)
    count+=1

print(testFileName)

#학습용 데이터 전체 읽기 (git 데이터)
for fn in trainFileName:
    print(str(fn))
    try:
        inData, outData = readData("git_input_data/"+str(fn))
    except:
        #위에서 예외가 발생한다는건, 이 파일이 git_input_data가 아닌 uci-preprocessed에 있다는 뜻임
        inData, outData = readData("uci-preprocessed/"+str(fn))
    trainData_in_list.append(inData)
    trainData_out_list.append(outData)

print("trainData_in_list_len: "+str(len(trainData_in_list)))
print("trainData_out_list_len: "+str(len(trainData_out_list)))

#시험용 데이터 전체 읽기 (git 데이터)
for fn in testFileName:
    print(str(fn))
    try:
        inData, outData = readData("git_input_data/"+str(fn))
    except:
        #위에서 예외가 발생한다는건, 이 파일이 git_input_data가 아닌 uci-preprocessed에 있다는 뜻임
        inData, outData = readData("uci-preprocessed/"+str(fn))
    testData_in_list.append(inData)
    testData_out_list.append(outData)

end_time = time.time()

print("File I/O elapsed time: " + str(end_time - start_time))

['preprocessed-46.csv', 'preprocessed-59.csv', 'preprocessed-45.csv', 'preprocessed-12.csv', 'preprocessed-34.csv', 'preprocessed-16.csv', 'preprocessed-48.csv', 'preprocessed-14.csv', 'preprocessed-50.csv', 'preprocessed-61.csv', 'preprocessed-43.csv', 'preprocessed-29.csv', 'preprocessed-15.csv', 'preprocessed-17.csv']
preprocessed-33.csv
preprocessed-5.csv
preprocessed-26.csv
preprocessed-38.csv
preprocessed-25.csv
preprocessed-70.csv
preprocessed-19.csv
preprocessed-53.csv
preprocessed-36.csv
preprocessed-69.csv
preprocessed-62.csv
preprocessed-42.csv
preprocessed-23.csv
preprocessed-18.csv
preprocessed-4.csv
preprocessed-1.csv
preprocessed-27.csv
preprocessed-31.csv
preprocessed-37.csv
preprocessed-58.csv
preprocessed-39.csv
preprocessed-65.csv
preprocessed-49.csv
preprocessed-57.csv
preprocessed-22.csv
preprocessed-10.csv
preprocessed-32.csv
preprocessed-52.csv
preprocessed-64.csv
preprocessed-35.csv
preprocessed-8.csv
preprocessed-51.csv
preprocessed-24.csv
preprocessed-7.csv
pr

In [5]:
print(uciFileList)

['preprocessed-33.csv', 'preprocessed-5.csv', 'preprocessed-26.csv', 'preprocessed-38.csv', 'preprocessed-25.csv', 'preprocessed-70.csv', 'preprocessed-19.csv', 'preprocessed-53.csv', 'preprocessed-36.csv', 'preprocessed-69.csv', 'preprocessed-62.csv', 'preprocessed-42.csv', 'preprocessed-23.csv', 'preprocessed-18.csv', 'preprocessed-4.csv', 'preprocessed-1.csv', 'preprocessed-27.csv', 'preprocessed-31.csv', 'preprocessed-37.csv', 'preprocessed-58.csv', 'preprocessed-39.csv', 'preprocessed-65.csv', 'preprocessed-49.csv', 'preprocessed-57.csv', 'preprocessed-22.csv', 'preprocessed-10.csv', 'preprocessed-32.csv', 'preprocessed-52.csv', 'preprocessed-64.csv', 'preprocessed-35.csv', 'preprocessed-8.csv', 'preprocessed-51.csv', 'preprocessed-24.csv', 'preprocessed-7.csv', 'preprocessed-44.csv', 'preprocessed-55.csv', 'preprocessed-13.csv', 'preprocessed-67.csv', 'preprocessed-47.csv', 'preprocessed-11.csv', 'preprocessed-2.csv', 'preprocessed-3.csv', 'preprocessed-40.csv', 'preprocessed-54.

In [6]:

# input place holders
X = tf.placeholder(tf.float32, [None, 7, 1])
Y = tf.placeholder(tf.float32, [None, 1])


#layer = {'weights':tf.Variable(tf.random_normal([rnn_size, 1])),'biases':tf.Variable(tf.random_normal([1]))}


# build a LSTM network
cell = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_size, state_is_tuple=True, activation=tf.nn.relu)
#cell = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_size, state_is_tuple=True, activation=tf.tanh)

outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=None)
# We use the last cell's output

# cost/loss
#loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
loss = tf.reduce_mean(tf.square(Y_pred - Y))
# optimizer
optimizer = tf.train.AdamOptimizer(0.01)
train = optimizer.minimize(loss)

# RMSE
targets = tf.placeholder(tf.float32, [None, 1])
predictions = tf.placeholder(tf.float32, [None, 1])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))

sess = tf.InteractiveSession()
tf.initialize_all_variables().run()
init = tf.global_variables_initializer()
sess.run(init)


Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [7]:
start_time = time.time()
# Training step

lossList = []
for loop in range(len(trainData_in_list)):
    losss = []
    for i in range(NUM_EPOCHS):
        _, step_loss = sess.run([train, loss],
                                feed_dict={X: trainData_in_list[loop], Y: trainData_out_list[loop]})
        losss.append(step_loss)
    print(str(loop / len(trainFileName) * 100) + "%")
    lossList.append(losss)
print("DONE")
end_time = time.time()
print("ML elapsed time: " + str(end_time - start_time))

0.0%
1.7857142857142856%
3.571428571428571%
5.357142857142857%
7.142857142857142%
8.928571428571429%
10.714285714285714%
12.5%
14.285714285714285%
16.071428571428573%
17.857142857142858%
19.642857142857142%
21.428571428571427%
23.214285714285715%
25.0%
26.785714285714285%
28.57142857142857%
30.357142857142854%
32.142857142857146%
33.92857142857143%
35.714285714285715%
37.5%
39.285714285714285%
41.07142857142857%
42.857142857142854%
44.642857142857146%
46.42857142857143%
48.214285714285715%
50.0%
51.78571428571429%
53.57142857142857%
55.35714285714286%
57.14285714285714%
58.92857142857143%
60.71428571428571%
62.5%
64.28571428571429%
66.07142857142857%
67.85714285714286%
69.64285714285714%
71.42857142857143%
73.21428571428571%
75.0%
76.78571428571429%
78.57142857142857%
80.35714285714286%
82.14285714285714%
83.92857142857143%
85.71428571428571%
87.5%
89.28571428571429%
91.07142857142857%
92.85714285714286%
94.64285714285714%
96.42857142857143%
98.21428571428571%
DONE
ML elapsed time: 449

In [8]:
e=[] #(실제값-예측값)^2 이 담길 리스트
rmseList=[] # 테스트케이스별 RMSE 값들이 담길 리스트
for j in range(len(testData_in_list)):
    for i, inputPoint in enumerate(testData_in_list[j]) :
        
        predicted = sess.run(prediction, feed_dict={x: [inputPoint]})
        predictedList.append(predicted[0][0])
        desired = testData_out_list[j][i][0]
        desiredList.append(desired)
        
        e.append( math.pow((predicted[0][0]-desired), 2) )
        
        #실제로 고혈당
        if(desired > 180):
            desiredHyper += 1
            #실제로 고혈당이면서 예측도 성공한 경우
            if(predicted[0][0] > 180):
                predictedHyper+=1
            elif(abs(desired - predicted[0][0]) > 8):
                falseHigh+=1
            
        if(desired<70):
            desiredHypo +=1
            #실제로 저혈당이면서 예측도 성공한 경우
            if(predicted[0][0]<70):
                predictedHypo+=1
            elif(abs(desired - predicted[0][0]) > 8):
                falseLow+=1
        
    print("desiredHyper: "+str(desiredHyper)+", predictedHyper: "+
          str(predictedHyper)+", desiredHypo: "+str(desiredHypo)+", predictedHypo: "+str(predictedHypo)+
         ", falseHigh: "+str(falseHigh)+", falseLow: "+str(falseLow))
    
    predictedHyperList.append(predictedHyper)
    predictedHypoList.append(predictedHypo)
    
    desiredHyperList.append(desiredHyper)
    desiredHypoList.append(desiredHypo)
    
    falseLowList.append(falseLow)
    falseHighList.append(falseHigh)
    
    
    predictedHyper = 0
    predictedHypo = 0

    desiredHyper = 0
    desiredHypo = 0
    
    falseLow = 0
    falseHigh = 0
    
    avg = sum(e) / len(e)
    rmse = math.sqrt(avg)
    rmseList.append(rmse)
    
    
    print("Patient: "+str(testFileName[j])+", RMSE: "+str(rmse))
    #실제-예측 데이터를 텍스트 파일로 저장 (나중에 엑셀같은걸로 차트만들때 편하라고)
    realFp = open("chartData/git+uci/RNN/PH"+str(PH)+"/"+testFileName[j]+"-real.txt", "w")
    predictFp = open("chartData/git+uci/RNN/PH"+str(PH)+"/"+testFileName[j]+"-predict.txt", "w")

    #고/저혈당 실제-예측 횟수 데이터 텍스트 파일로 저장
    predictedHyperFp = open("chartData/git+uci/RNN/PH"+str(PH)+"/"+testFileName[j]+"-predictedHyperList.txt", "w")
    desiredHyperFp = open("chartData/git+uci/RNN/PH"+str(PH)+"/"+testFileName[j]+"-desiredHyperList.txt", "w")
    
    predictedHypoFp = open("chartData/git+uci/RNN/PH"+str(PH)+"/"+testFileName[j]+"-predictedHypoList.txt", "w")
    desiredHypoFp = open("chartData/git+uci/RNN/PH"+str(PH)+"/"+testFileName[j]+"-desiredHypoList.txt", "w")

    falseHighFp = open("chartData/git+uci/RNN/PH"+str(PH)+"/"+testFileName[j]+"-falseHigh.txt", "w")
    falseLowFp =open("chartData/git+uci/RNN/PH"+str(PH)+"/"+testFileName[j]+"-falseLow.txt", "w")

    #실제 혈당 저장
    for data in desiredList:
        realFp.write(str(data)+'\n')

    #예측 혈당 저장
    for data in predictedList:
        predictFp.write(str(data)+'\n')

    #고혈당 예측한 횟수 저장
    for data in predictedHyperList:
        predictedHyperFp.write(str(data)+"\n")

    #고혈당 실제 횟수 저장
    for data in desiredHyperList:
        desiredHyperFp.write(str(data)+"\n")

    #저혈당 예측한 횟수 저장
    for data in predictedHypoList:
        predictedHypoFp.write(str(data)+"\n")

    #저혈당 실제 횟수 저장
    for data in desiredHypoList:
        desiredHypoFp.write(str(data)+"\n")

    #잘못된 고혈당 예측 횟수 저장
    for data in falseHighList:
        falseHighFp.write(str(data)+"\n")
        
    #잘못된 저혈당 예측 횟수 저장
    for data in falseLowList:
        falseLowFp.write(str(data)+"\n")
        
    realFp.close()
    predictFp.close()
    predictedHyperFp.close()
    desiredHyperFp.close()
    predictedHypoFp.close()
    desiredHypoFp.close()
    falseHighFp.close()
    falseLowFp.close()
    
    predictedList=[]
    desiredList=[]
    e=[]

NameError: name 'prediction' is not defined

[]
